[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the EECS JupyterHub as well as in the binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [33]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [34]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

In [28]:
filters1= [
    'areaType=overview'
]

structure1= {
    "date":"date",
    "NewCases":"newCasesByPublishDate",
    "CurriedTests":"newTestsByPublishDate",
    "PlanedTested":"plannedCapacityByPublishDate"
}
api2 = Cov19API (filters=filters1, structure=structure1)
testsdistribution=api2.get_json()
with open("testsdistribution.json", "wt") as OUTF:
    json.dump(testsdistribution, OUTF)


## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

In [29]:
with open("testsdistribution.json", "rt") as INFIFLE1:
    data1 =json.load(INFIFLE1)
datalist1 =data1['data']
dates1 = [dictionary ['date'] for dictionary in datalist1 ]
dates1.sort()
def parse_date1 (datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")
startdate1 =parse_date1(dates1[0])
enddate1 =parse_date1(dates1[-1])
index1 =pd.date_range(startdate1, enddate1, freq='D')
timeseriesdf1=pd.DataFrame(index=index1, columns=['NewCases','CurriedTests', 'PlanedTested'])
for entry in datalist1: 
    dates1=parse_date1(entry['date'])
    for column in ['NewCases','CurriedTests', 'PlanedTested']:
        if pd.isna(timeseriesdf1.loc[dates1, column]): 
            value1= float(entry[column]) if entry[column]!=None else 0.0
            timeseriesdf1.loc[dates1, column]=value1
timeseriesdf1.fillna(0.0, inplace=True)
timeseriesdf1.to_pickle("timeseriesdf1.pkl")



## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [23]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 


In [30]:

def access_api1(button):
    # Ignore the parameter, put code for polling the API here
    print("I'm downloading data from the API...")
    print("...all done.")
apibutton1=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)



Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [32]:
apibutton1.on_click(access_api1)
display(apibutton1)
timeseriesdf1=pd.read_pickle("timeseriesdf1.pkl")

series1=wdg.SelectMultiple(
    options=['NewCases', 'CurriedTests', 'PlanedTested'],
    value=['NewCases', 'CurriedTests', 'PlanedTested'],
    rows=3,
    description='Stats:',
    disabled=False
)

scale1=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls1=wdg.HBox([series1, scale1])

def timeseries1_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        timeseriesdf1[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

graph1=wdg.interactive_output(timeseries1_graph, {'gcols': series1, 'gscale': scale1})

ctrls1=wdg.VBox([series1, scale1])
form1=wdg.HBox([graph1, ctrls1])
display(form1)


Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*